In [1]:
# Import necessary packages
import pandas as pd
import numpy as np
import geopandas as gp
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
import dask.distributed
client = dask.distributed.Client()

import warnings
warnings.filterwarnings("ignore")

In [2]:
#Read in the INRIX data using dask
filename1 = 'Z:/SF_CMP/Data/SF_all_rds_2019-02-11_to_2019-03-24_1_min_part_1/data.csv'
df1 = dd.read_csv(filename1, assume_missing=True)

filename2 = 'Z:/SF_CMP/Data/SF_all_rds_2019-02-11_to_2019-03-24_1_min_part_2/data.csv'
df2 = dd.read_csv(filename2, assume_missing=True)

filename3 = 'Z:/SF_CMP/Data/SF_all_rds_2019-02-11_to_2019-03-24_1_min_part_3/data.csv'
df3 = dd.read_csv(filename3, assume_missing=True)

print(('Part 1 record #: %s, Part 2 record #: %s, Part 3 record #: %s') % (len(df1), len(df2), len(df3)))

df1.head(2)

Part 1 record #: 22264750, Part 2 record #: 16822195, Part 3 record #: 4794375


,Date Time,Segment ID,Speed(miles/hour),Hist Av Speed(miles/hour),Ref Speed(miles/hour),Travel Time(Minutes),CValue,Pct Score30,Pct Score20,Pct Score10,Road Closure,Corridor/Region Name
0,2019-02-12T00:09:00-08:00,169786058.0,67.0,63.0,60.0,0.5,98.0,100.0,0.0,0.0,F,SF all rds
1,2019-02-12T00:10:00-08:00,169786058.0,67.0,63.0,60.0,0.5,100.0,100.0,0.0,0.0,F,SF all rds


In [3]:
#Combine three data parts
df = dd.concat([df1,df2],axis=0,interleave_partitions=True)
df = dd.concat([df,df3],axis=0,interleave_partitions=True)
print('Combined record #: ', len(df))

Combined record #:  43881320


In [4]:
#Create date and time fields for subsequent filtering
df['Date_Time'] = df['Date Time'].str[:16]
df['Date_Time'] = df['Date_Time'].str.replace('T', " ")
df['Date'] = df['Date_Time'].str[:10]
df['Day']=dd.to_datetime(df['Date_Time'])
df['DOW']=df.Day.dt.dayofweek #Tue-1, Wed-2, Thu-3
df['Hour']=df.Day.dt.hour
df['Minute']=df.Day.dt.minute

In [5]:
#Get AM (7-9am) speeds on Tue, Wed, and Thu
df_am=df[((df['DOW']>=1) & (df['DOW']<=3)) & ((df['Hour']==7) | (df['Hour']==8))]
print('Number of records in AM: ', len(df_am))

Number of records in AM:  5346041


In [6]:
#Get PM (4:30-6:30pm) speeds on Tue, Wed, and Thu
df_pm=df[((df['DOW']>=1) & (df['DOW']<=3)) & (((df['Hour']==16) & (df['Minute']>=30)) | (df['Hour']==17) | ((df['Hour']==18) & (df['Minute']<30)))]
print('Number of records in PM: ', len(df_pm))

Number of records in PM:  5683124


In [7]:
#Calculate AM sample size for each XD link
df_am_sum=df_am.groupby('Segment ID').Date_Time.agg(['count']).compute()
df_am_sum=df_am_sum.reset_index()
df_am_sum.columns = ['Segment ID','Count_AM']

In [8]:
#Calculate PM sample size for each XD link
df_pm_sum=df_pm.groupby('Segment ID').Date_Time.agg(['count']).compute()
df_pm_sum=df_pm_sum.reset_index()
df_pm_sum.columns = ['Segment ID','Count_PM']

In [9]:
num_days=18 #six weeks
df_am_sum['Pcnt_AM']=round(100*df_am_sum['Count_AM']/num_days/2/60,3)
df_pm_sum['Pcnt_PM']=round(100*df_pm_sum['Count_PM']/num_days/2/60,3)
print('Number of links with speeds in AM: ', len(df_am_sum))
print('Number of links with speeds in PM: ', len(df_pm_sum))

Number of links with speeds in AM:  4144
Number of links with speeds in PM:  4161


In [10]:
#Read in INRIX XD network shapefile
shp_path='Z:/SF_CMP/Data/inrixXD_SF/inrixXD_SF.shp'
sf_shp = gp.read_file(shp_path)
print('Number of links in SF: ', len(sf_shp))
sf_shp.head(2)

Number of links in SF:  4218


,OID_1,XDSegID,PreviousXD,NextXDSegI,FRC,RoadNumber,RoadName,LinearID,Country,State,...,SpecialRoa,RoadList,StartLat,StartLong,EndLat,EndLong,Bearing,XDGroup,ShapeSRID,geometry
0,None,168424587,1626689930,1626704693,2,None,Mission Street,None,United States of America,California,...,None,Mission Street,37.76344,-122.41953,37.76505,-122.41969,N,1303785,4326,LINESTRING (-122.4195301579087 37.763439720435...
1,None,168716200,None,None,4,None,Hayes Street,None,United States of America,California,...,None,Hayes Street,37.77668,-122.42465,37.77671,-122.42438,E,1310140,4326,LINESTRING (-122.424650108766 37.7766802021228...


In [11]:
#Attach the sample size measures to the shapefile
df_am_sum['Segment ID']=df_am_sum['Segment ID'].astype(np.int64)
df_pm_sum['Segment ID']=df_pm_sum['Segment ID'].astype(np.int64)
sf_shp['XDSegID']=sf_shp['XDSegID'].astype(np.int64)
df_samplesize=pd.merge(sf_shp, df_am_sum, left_on='XDSegID', right_on='Segment ID', how='left') #AM
df_samplesize=pd.merge(df_samplesize, df_pm_sum, left_on='XDSegID', right_on='Segment ID', how='left') #PM

In [12]:
#Create adequacy indicators for 10%, 30%, and 50% thresholds
df_samplesize['ADQT_AM_10']=df_samplesize.apply(lambda x: 'Yes' if x['Pcnt_AM']>10 else 'No', axis=1) #10% threshold for AM
df_samplesize['ADQT_AM_30']=df_samplesize.apply(lambda x: 'Yes' if x['Pcnt_AM']>30 else 'No', axis=1) #30% threshold for AM
df_samplesize['ADQT_AM_50']=df_samplesize.apply(lambda x: 'Yes' if x['Pcnt_AM']>50 else 'No', axis=1) #50% threshold for AM

df_samplesize['ADQT_PM_10']=df_samplesize.apply(lambda x: 'Yes' if x['Pcnt_PM']>10 else 'No', axis=1) #10% threshold for PM
df_samplesize['ADQT_PM_30']=df_samplesize.apply(lambda x: 'Yes' if x['Pcnt_PM']>30 else 'No', axis=1) #30% threshold for PM
df_samplesize['ADQT_PM_50']=df_samplesize.apply(lambda x: 'Yes' if x['Pcnt_PM']>50 else 'No', axis=1) #50% threshold for PM

In [13]:
df_samplesize.head()

,OID_1,XDSegID,PreviousXD,NextXDSegI,FRC,RoadNumber,RoadName,LinearID,Country,State,...,Pcnt_AM,Segment ID_y,Count_PM,Pcnt_PM,ADQT_AM_10,ADQT_AM_30,ADQT_AM_50,ADQT_PM_10,ADQT_PM_30,ADQT_PM_50
0,None,168424587,1626689930,1626704693,2,None,Mission Street,None,United States of America,California,...,65.000,168424587.0,703.0,32.546,Yes,Yes,Yes,Yes,Yes,No
1,None,168716200,None,None,4,None,Hayes Street,None,United States of America,California,...,68.380,168716200.0,1462.0,67.685,Yes,Yes,Yes,Yes,Yes,Yes
2,None,168746767,None,None,4,None,Letterman Drive,None,United States of America,California,...,51.343,168746767.0,1171.0,54.213,Yes,Yes,Yes,Yes,Yes,Yes
3,None,168751741,None,None,4,None,Letterman Drive,None,United States of America,California,...,51.343,168751741.0,1171.0,54.213,Yes,Yes,Yes,Yes,Yes,Yes
4,None,168751736,None,None,4,None,Battery Caulfield Road,None,United States of America,California,...,48.750,168751736.0,617.0,28.565,Yes,Yes,No,Yes,No,No


In [14]:
#Output the combined shapefile
df_samplesize.to_file('Z:/SF_CMP/Data/SF_INRIX_Sample_Size_SixWeeks.shp')